# 모델 매개변수 최적화 하기

## 기본코드

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


## 하이퍼파라미터
- 에폭(epoch) 수 - 데이터셋을 반복하는 횟수
- 배치크기(batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- 학습률(lr) - 각 배치/애폭에서 모델의 매개변수를 조절하는 비율. 값이 작은 수록 학습속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.

In [5]:
lerning_rate = 1e-3
batch_size = 64
epoch = 5

## 최적화 단계
- 학습 단계: 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴합니다.
- 검증/테스트 단계: 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복합니다.

## 손실함수
학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높습니다. **손실함수**는 획득한 결과와 실제 값 사이의 틀린 정도를 측정하며, 학습 중에 이 값을 최소화하려고합니다. 주어진 데이터 샘플을 입력으로 계산한 예측과 정답을 비교하여 손실을 계산합니다.

In [6]:
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저
최적화는 각 학습단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다. **최적화 알고리즘**은 이 과정이 수행되는 방식

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=lerning_rate)

## 전체 구현

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch & 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fun):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lerning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done")

Epoch 1

loss: 2.294700 [   64/60000]
loss: 2.300138 [  128/60000]
loss: 2.315917 [  192/60000]
loss: 2.309500 [  256/60000]
loss: 2.303650 [  576/60000]
loss: 2.315490 [  640/60000]
loss: 2.310088 [  704/60000]
loss: 2.306830 [  768/60000]
loss: 2.291355 [ 1088/60000]
loss: 2.293030 [ 1152/60000]
loss: 2.303245 [ 1216/60000]
loss: 2.297059 [ 1280/60000]
loss: 2.289589 [ 1600/60000]
loss: 2.291426 [ 1664/60000]
loss: 2.306644 [ 1728/60000]
loss: 2.304031 [ 1792/60000]
loss: 2.277407 [ 8256/60000]
loss: 2.282192 [ 8320/60000]
loss: 2.274353 [ 8384/60000]
loss: 2.276133 [ 8448/60000]
loss: 2.283121 [ 8768/60000]
loss: 2.279510 [ 8832/60000]
loss: 2.293397 [ 8896/60000]
loss: 2.278743 [ 8960/60000]
loss: 2.285091 [ 9280/60000]
loss: 2.276168 [ 9344/60000]
loss: 2.280649 [ 9408/60000]
loss: 2.294257 [ 9472/60000]
loss: 2.289640 [ 9792/60000]
loss: 2.281523 [ 9856/60000]
loss: 2.278614 [ 9920/60000]
loss: 2.275810 [ 9984/60000]
loss: 2.274391 [16448/60000]
loss: 2.253096 [16512/60000]
loss: